In [1]:
from langchain.chat_models import ChatOpenAI

In [2]:
import sqlite3

In [3]:
import pandas as pd

In [6]:
import os

In [4]:
OPENAI_API_KEY = '****'

In [7]:
os.environ['OPENAI_API_KEY'] = '*'

In [8]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo-1106')

In [9]:
connection = sqlite3.connect("customer.db")

In [12]:
df = pd.read_csv("/Users/madhavathavale/Library/Mobile Documents/com~apple~CloudDocs/workspace/python/retail_sales_dataset.csv")

In [13]:
df

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100
...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,CUST996,Male,62,Clothing,1,50,50
996,997,2023-11-17,CUST997,Male,52,Beauty,3,30,90
997,998,2023-10-29,CUST998,Female,23,Beauty,4,25,100
998,999,2023-12-05,CUST999,Female,36,Electronics,3,50,150


In [14]:
df.to_sql("RetailSalesTable", connection, if_exists= 'replace')

1000

In [16]:
from langchain.agents import create_sql_agent

In [17]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

In [18]:
from langchain.agents.agent_types import AgentType

In [19]:
from langchain.sql_database import SQLDatabase

In [27]:
db = SQLDatabase.from_uri("sqlite:///customer.db")

In [34]:
agent_executor = create_sql_agent(llm=llm, toolkit = SQLDatabaseToolkit(db=db, llm=llm),
                                 agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                                 handle_parsing_errors = True, verbose=True, return_intermediate_steps= True)

In [32]:
user_inquiry = "What is the number of transactions per year and month?"

In [35]:
user_inquiry = "What is the number of transactions across different age groups of customers?"

In [36]:
agent_executor.run(user_inquiry)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: RetailSalesTable
Thought:I need to check the schema of the RetailSalesTable to see if it contains information about customer age groups and transactions.
Action: sql_db_schema
Action Input: RetailSalesTable
Observation: 
CREATE TABLE "RetailSalesTable" (
	"index" INTEGER, 
	"Transaction ID" INTEGER, 
	"Date" TEXT, 
	"Customer ID" TEXT, 
	"Gender" TEXT, 
	"Age" INTEGER, 
	"Product Category" TEXT, 
	"Quantity" INTEGER, 
	"Price per Unit" INTEGER, 
	"Total Amount" INTEGER
)

/*
3 rows from RetailSalesTable table:
index	Transaction ID	Date	Customer ID	Gender	Age	Product Category	Quantity	Price per Unit	Total Amount
0	1	2023-11-24	CUST001	Male	34	Beauty	3	50	150
1	2	2023-02-27	CUST002	Female	26	Clothing	2	500	1000
2	3	2023-01-13	CUST003	Male	50	Electronics	1	30	30
*/
Thought:I can use the "Age" column from the RetailSalesTable to calculate the number of transactions across different age groups of 

'The number of transactions across different age groups of customers are as follows:\nAge 18: 21 transactions\nAge 19: 21 transactions\nAge 20: 21 transactions\nAge 21: 20 transactions\nAge 22: 27 transactions\nAge 23: 24 transactions\nAge 24: 15 transactions\nAge 25: 20 transactions\nAge 26: 22 transactions\nAge 27: 23 transactions'

In [37]:
from langchain_core.prompts import PromptTemplate

In [38]:
template = PromptTemplate(

    input_variables = ['user_inquiry', 'background_info'],
    template = """{background_info} Question: {user_inquiry}"""
)

In [39]:
background_info ="""As the customer analyst, my role is to analyze the transaction patterns of customers. The feature engineering in table 'RetailSalesTable' is crucial for statistical exploration. For example:
- column 'Age' can be grouped into bins of age ranges, such as 21-25, 26-30, and so on.
Understanding the data in these columns helps us gain insights about our customers, enabling us to offer personalized services and develop effective marketing strategies.
"""

In [40]:
user_inquiry = "What is the number of transactions across different age ranges of customers?"


In [41]:
agent_executor.run(template.format(background_info =background_info, user_inquiry=user_inquiry))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: RetailSalesTable
Thought:I need to check the schema of the RetailSalesTable to see the columns available for my query.
Action: sql_db_schema
Action Input: RetailSalesTable
Observation: 
CREATE TABLE "RetailSalesTable" (
	"index" INTEGER, 
	"Transaction ID" INTEGER, 
	"Date" TEXT, 
	"Customer ID" TEXT, 
	"Gender" TEXT, 
	"Age" INTEGER, 
	"Product Category" TEXT, 
	"Quantity" INTEGER, 
	"Price per Unit" INTEGER, 
	"Total Amount" INTEGER
)

/*
3 rows from RetailSalesTable table:
index	Transaction ID	Date	Customer ID	Gender	Age	Product Category	Quantity	Price per Unit	Total Amount
0	1	2023-11-24	CUST001	Male	34	Beauty	3	50	150
1	2	2023-02-27	CUST002	Female	26	Clothing	2	500	1000
2	3	2023-01-13	CUST003	Male	50	Electronics	1	30	30
*/
Thought:I can use the "Age" column to group transactions into different age ranges.
Action: sql_db_query
Action Input: SELECT CASE 
                    WHEN Age BETWEE

'The number of transactions across different age ranges of customers are:\n- 21-25: 106 transactions\n- 26-30: 104 transactions\n- 31-35: 101 transactions\n- 36-40: 92 transactions\n- Over 40: 597 transactions'

In [42]:
from langchain import LLMChain


In [43]:
from langchain.agents import (AgentExecutor, Tool, ZeroShotAgent)

In [44]:
from langchain_experimental.tools import PythonREPLTool

In [45]:
description = (
    "Useful when you require to answer analytical questions about customers. "
    "Use this more than the Python REPL tool if the question is about customer analytics,"
    "like 'How many customers are there?' or 'count the number of transactions by age group'. "
    "Try not to use clause in the SQL."
)

In [48]:
customer_data_tool = Tool(name="Customer", func=agent_executor.run, description=description)

In [49]:
tools = [PythonREPLTool()]

In [50]:
tools.append(customer_data_tool)

In [51]:
prefix = "Below are tools that you can access:"

In [52]:
suffix = (
    "Pass the relevant part of the request directly to the Customer tool.\n\n"
    "Request: {input}\n"
    "{agent_scratchpad}"
)

In [55]:
agent_prompt = ZeroShotAgent.create_prompt(tools, prefix=prefix, suffix=suffix, 
                                           input_variables=['input', 'agent_scratchpad'])

In [56]:
zero_shot_agent = ZeroShotAgent(llm_chain = LLMChain(llm=llm, prompt=agent_prompt),
                                           allowed_tools = [tool.name for tool in tools])

In [62]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=zero_shot_agent, tools = tools, verbose=False, 
                                                    handle_parsing_errors=True)

In [63]:
user_inquiry = "Use a grouped bar graph to visualize the result of the following inquiry: What are the relationships between product category, average total amount, and gender?"

In [64]:
agent_executor.run(user_inquiry)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: RetailSalesTable
Thought:

OutputParserException: Could not parse LLM output: `I don't know.`